In [50]:
import boto3
import os
import pandas as pd
import time
import spacy
from quotes import quote_extractor

In [40]:
year = 2011
paper = "scmp"
bucket = "newyorktime"
key = f"{paper}/{year}.csv"

In [48]:
s3_client = boto3.client("s3")
nlp = spacy.load("en_core_web_lg")

In [16]:
os.makedirs("data", exist_ok=True) 

In [17]:
filepath = os.path.join('data', key)

In [18]:
os.path.dirname(filepath) and os.makedirs(os.path.dirname(filepath), exist_ok=True) 


In [34]:
s3_client.download_file(bucket, key, filepath)

df = pd.read_csv(filepath)

In [36]:
def timeit(fn, *args, **kwargs):
    s = time.perf_counter()
    ret = fn(*args, **kwargs)
    e = time.perf_counter()
    print(f"{fn.__name__} took {e-s} secs")
    return ret

In [57]:
def extract_quotes(row, text_col, uid_col, publication="scmp", year="2012"):
    """Row is a row in a dataframe with a text column and unique idenifier column."""
    doc = nlp(row[text_col])
    # list of dictionaries
    final_quotes = quote_extractor.extract_quotes(row[uid_col], doc)
    for item in final_quotes:
        item["source"] = row[uid_col]
        item["publication"] = publication
        item["year"] = year
    return final_quotes

In [58]:
def run(input_df, output_name, publication="scmp", year="2012"):
    quote_dcts = input_df.apply(
        lambda row: extract_quotes(row, "Body", "Index", publication, year), axis=1
    )
    quotedf = pd.json_normalize(quote_dcts.explode()).dropna().convert_dtypes()
    quotedf.to_csv(output_name)
    return quotedf

In [59]:
def upload_s3(filepath, bucket, key=None):
    if not key:
        key = filepath
    try:
        response = s3_client.upload_file(filepath, bucket, key)
        return response
    except ClientError as e:
        logging.exception(e)


In [60]:
# quotedf = timeit(run, df, path)
# upload_s3(path, bucket, key=f"{paper}/quotes/q_{year}.csv")    

In [61]:
from tqdm import tqdm

In [ ]:
for year in tqdm(range(2012, 2021)):
    key = f"{paper}/{year}.csv"
    filepath = os.path.join('data', key)    
    path = f"quotes_{paper}_{year}.csv"
    print("working on ", key)
    s3_client.download_file(bucket, key, filepath)
    df = pd.read_csv(filepath)
    df["Body"] = df.Body.astype(str)
    quotedf = timeit(run, df, path, paper, year)
    upload_s3(path, bucket, key=f"{paper}/quotes/q_{year}.csv")    

  0%|          | 0/9 [00:00<?, ?it/s]

working on  scmp/2012.csv
